##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hello Worldの先へ、コンピュータビジョンの例

前の練習問題では、解決しようとしている問題を解くニューラルネットワークの作成方法を見ました。これにより、学習した動作の例がはっきりわかりました。もちろん、その例では、固定した値の集合についてXとYの関係を学習するために、機械学習を使用して、それをすべての値に拡張するという面倒な方法ではなく、関数Y=2x-1を直接書く方が簡単だったため、少しやり過ぎでした。

しかし、はるかに複雑な、たとえばコンピュータビジョン問題のようなルールを書くシナリオではどうでしょうか。10種類のタイプがあるデータセットで訓練され、さまざまなファッションアイテムを認識するシナリオを見てみましょう。

## コーディングを開始する

TensorFlowのインポートから始めましょう

In [ ]:
import tensorflow as tf
print(tf.__version__)

Fashion MNISTデータは、tf.kerasデータセットAPIで直接入手できます。これを、次のようにしてロードします。

In [ ]:
mnist = tf.keras.datasets.fashion_mnist

このオブジェクトに対してload_dataを呼び出すと、２つのリストの２つのセットを取得できます。これらは、ファッションアイテムとそれらのラベルを含んだグラフィックの訓練用とテスト用の値です。


In [ ]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

これらの値は、どのように見えるでしょうか。訓練用画像と訓練用ラベルを表示して見てみましょう。配列の中のさまざまなインデックスで実験してみましょう。たとえば、インデックス42も見てみると、これはインデックス０とは別のブーツです。


In [ ]:
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])

数値のすべての値が０から255までであることに気づきます。ニューラルネットワークを訓練する場合、さまざまな理由から、扱う値のすべてが０と１の間の方が簡単です。このプロセスを「**正規化**」といいますが、幸いPythonでは、このようなリストをループなしで容易に正規化できます。次のようにして行います。


In [ ]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

訓練用とテスト用の２つのセットがあるのを疑問に思われるかもしれません。これについて最初にお話ししたことを覚えていますか？考え方としては、訓練用のデータセットが１つあり、もう１つのデータセットは、モデルがまだ見ていないデータであり、どのくらいうまく値を分類できるかを見るためのものです。結局のところ、訓練が全て終わると、まだ見ていないデータで試したくなります。

では、モデルを設計しましょう。ここには、新しい概念がかなりたくさんありますが、心配いりません。すぐにこつをつかめます。 

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

**Sequential**：これはニューラルネットワーク内の層の順序（SEQUENCE）を定義します。

**Flatten**：以前、画像が正方形のとき、これを表示したことを覚えていますか？Flattenはこの正方形を取り、1次元のセットに変換します。


**Dense**：ニューロンの層を追加します。

各ニューロン層に、何をすべきかを教える**活性化関数**が必要です。多くのオプションがありますが、今のところは、これらを使用していきましょう。 

**Relu** は、「X>0の場合はXを返し、そうでない場合は０を返す」ことを意味します。したがって、ゼロかゼロより大きい値だけをネットワーク内の次の層に渡します。

**Softmax** は、一連の値を取り、最大のものを選びます。たとえば、最後の層の出力が[0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05]の場合、最大の値をいちいち探す手間を省いて、[0,0,0,0,1,0,0,0,0]に変換します。コーディングの手間を省くのが目的です。

モデルが定義できたので、次にすべきことは、実際にモデルを構築することです。このためには、以前と同じように、最適化アルゴリズムと損失関数でコンパイルしてから、**model.fit ** を呼び出し、訓練用データを訓練用ラベルに合わせるように指示することによって訓練します。つまり、訓練用データと実際のラベルの間の関係を見つけさせて、将来、訓練用データに似たデータが出てきた場合に、そのデータが何に似ているか予測できるようにします。

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

訓練が済んだら、最後のエポックの終わりに精度の値が表示されます。0.9098というような値です。これは、ニューラルネットワークが訓練用データの分類において約91%の精度であることを示しています。つまり、画像とラベルの間のパターンマッチを91%の確率で見つけ出しました。それほど良いわけではありませんが、わずか５エポックの訓練で、しかも非常に迅速に行ったことを考えると、それほど悪くありません。

しかし、見たことがないデータでは、どうでしょうか？それが、テスト画像がある理由です。model.evaluateを呼び出して、２つのセットを渡すと、それぞれの損失が報告されます。試してみましょう。


In [ ]:
model.evaluate(test_images, test_labels)

約0.8838の精度が返されました。これは約88%の精度であることを意味します。予測どおり、見たことがないデータでは、訓練したデータほどはうまくいかない可能性があります。この講座を進んでいく過程で、これを改善する方法を見ていきます。 

さらに探究するには、下記の練習問題に挑戦してみてください。


# 研究用練習問題

### 練習１：
この最初の練習問題では、下記のコードを実行します。それぞれのテスト画像ごとに分類のセットを作成し、分類の最初のエントリを表示します。実行後の出力は、数のリストです。このようになっているのはなぜだと思いますか？また、それらの数は何を表すのでしょうか？

In [ ]:
classifications = model.predict(test_images)

print(classifications[0])

ヒント：print(test_labels[0])を実行してみてください。９になります。このリストがそのようになっている理由を理解するのに役立つでしょうか？


In [ ]:
print(test_labels[0])

### このリストは何を表していますか？


1. 10個のランダムな意味のない値です
1. コンピューターが行った最初の10個の分類です
1. このアイテムが10クラスのそれぞれである確率です


#### 答え：
正解は(3)です

モデルの出力は、10個の数のリストです。これらの数は、分類される値が、対応する値 (https://github.com/zalandoresearch/fashion-mnist#labels) である確率です。 つまり、リストの最初の値は、画像が「０」（Tシャツ／トップス）である確率であり、次は「１」（ズボン）である確率です。すべて非常に低い確率であることに注目してください。

９（アンクルブーツ）の場合、確率は90台でした。つまり、ニューラルネットワークは、それがほぼ確実に７であると告げています。

### このリストからアイテムがアンクルブーツであることがどのようにしてわかりますか？


1.   情報不足なので、質問に答えられない
2.   リストの10番目の要素が最大であり、アンクルブーツのラベルは９である
2.   アンクルブーツのラベルは９であり、リストには０から９までの要素がある


#### 答え
正解は(2)です。リストとラベルは両方とも０から始まるので、アンクルブーツのラベルが９であるということは、10クラスのうちの10番目であることを意味します。リストの10番目の要素が最も高い値を持つということは、ニューラルネットワークは、分類しているアイテムがアンクルブーツである可能性が最も高いと予測したことを意味します。

## 練習２： 
今度は、モデル内の層を見てみましょう。512のニューロンを持つDense層について、さまざまな値で試してください。損失や訓練時間などの結果は、どのように異なりますか？そのようになるのは、なぜだと思いますか？


In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

### 質問1.ニューロンを1024に増やします。どのような影響がありますか？

1. 訓練時間は長くなるが、より正確になる
2. 訓練時間は長くなるが、精度は変わらない
3. 訓練時間は同じだが、より正確になる


#### 答え
正解は(1)です。ニューロンを追加することによって、より多くの計算を行わなければならず、プロセスは遅くなりますが、この場合、良い影響が得られます。つまり、より正確になります。常に「多い方が良い」わけではありません。すぐに収穫逓減の法則に突き当ります。

## 練習３：

Flatten()層を削除すると、どうなりますか。なぜそうなると思いますか？ 

データの形状に関するエラーになります。今はあいまいに見えるかもしれませんが、ネットワークの最初の層はデータと同じ形状であるべきであるという経験則を補強するものです。今のところ、私たちのデータは28x28の画像であり、28ニューロンの28層は実行不可能なため、28,28を784x1にフラット化する方が意味があります。それを処理するためのコードをすべて自分で書く代わりに、初めにFlatten()層を追加します。すると、後で配列がモデルにロードされるときに、自動的にフラット化されます。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([#tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

## 練習４：

最終（出力）層について考えてください。なぜ、10個なのでしょうか。10以外だとどうなるでしょうか？たとえば、５でネットワークを訓練してみてください。

予期しない値を見つけるとすぐにエラーになります。もう一つの経験則として、最後の層のニューロンの数は、分類しているクラスの数と一致すべきです。この例では、0-9の10個の数字なので、最終層には10個のニューロンが必要です。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

## 練習５：

ネットワークに追加する層の影響について考えてください。512の層と最後の10の層の間に別の層を追加した場合、どうなりますか？

答え：これは比較的単純なデータであるため、大きな影響はありません。はるかに複雑なデータの場合（次の授業で見ていく、花として分類されるカラー画像など）、追加の層が必要になることがよくあります。 


In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

# 練習６：


訓練のエポック数を増減した場合の影響について考えてください。なぜそうなると思いますか？

15エポックを試してください。おそらく、５エポックよりはるかに損失の少ないモデルになります。
30エポックを試してください。損失の値の減少がとまり、ときには増加するでしょう。これは、[どこかで]学習する「過学習」と呼ばれるものの副作用であり、ニューラルネットワークを訓練するときに注意が必要なものです。損失が改善しなければ、訓練に無駄な時間を費やしても意味がありません。

In [ ]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=30)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[34])
print(test_labels[34])

# 練習７：

訓練の前にデータを正規化し、0-255の値を0-1の値にしました。これをなくすと、どのような影響がありますか？以下の完全なコードで試してみてください。結果が変わるのはなぜだと思いますか？ 


In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

# 練習８：

以前、追加のエポックで訓練したとき、損失が変化する問題が発生しました。訓練時間が少し長くなり、「目的の値に達したところで訓練を中止できるといいのに」と思ったかもしれません。つまり、95%の精度で十分であり、３エポック後にその値に達した場合、さらに多くのエポックが終了するのをなぜ待たなければならないのでしょうか？では、これをどのように修正しますか？ほかのプログラムと同様です。つまり、コールバックするのです。その動作を見てみましょう。

In [ ]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.4):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])
